In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import math

In [ ]:
data_b_NO2=pd.read_excel("/Users/roshanchandraayyadevara/Documents/Dataset_Dessertation/year/NO2_Data.xlsx",sheet_name='Birmingham')
data_l_NO2=pd.read_excel("/Users/roshanchandraayyadevara/Documents/Dataset_Dessertation/year/NO2_Data.xlsx",sheet_name='London')
data_lp_NO2=pd.read_excel("/Users/roshanchandraayyadevara/Documents/Dataset_Dessertation/year/NO2_Data.xlsx",sheet_name='Liverpool')

In [ ]:
df_b_NO2=data_b_NO2[["Year","Site_Name","City","Annual_Mean"]]
df_l_NO2=data_l_NO2[["Year","Site_Name","City","Annual_Mean"]]
df_lp_NO2=data_lp_NO2[["Year","Site_Name","City","Annual_Mean"]]

In [ ]:
df_b_NO2=data_b_NO2.groupby('Year').agg(NO2_Birmingham=('Annual_Mean', 'mean')).reset_index()
df_l_NO2=data_l_NO2.groupby('Year').agg(NO2_London=('Annual_Mean', 'mean')).reset_index()
df_lp_NO2=data_lp_NO2.groupby('Year').agg(NO2_Liverpool=('Annual_Mean', 'mean')).reset_index()

In [ ]:
df_lp_NO2

In [ ]:
B_NO2=df_b_NO2['NO2_Birmingham']
l_NO2=df_l_NO2['NO2_London']
lp_NO2=df_lp_NO2['NO2_Liverpool']

In [ ]:
print("Maximum of Birmingham",max(B_NO2))
print("Maximum of London",max(l_NO2))
print("Maximum of Liverpool",max(lp_NO2))

In [ ]:
print("Minimum of Birmingham",min(B_NO2))
print("Minimum of London",min(l_NO2))
print("Minimum of Liverpool",min(lp_NO2))

In [ ]:
print("SD of Birmingham",B_NO2.std())
print("SD of London",l_NO2.std())
print("SD of Liverpool",lp_NO2.std())

In [ ]:
print("Variance of Birmingham",B_NO2.var())
print("Variance of London",l_NO2.var())
print("Variance of Liverpool",lp_NO2.var())

In [ ]:
print("Mean of Birmingham",B_NO2.mean())
print("Mean of London",l_NO2.mean())
print("Mean of Liverpool",lp_NO2.mean())

In [ ]:
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.multicomp as mc

# Assuming B_NO2, l_NO2, and lp_NO2 are pandas Series containing the data for Birmingham, London, and Liverpool respectively.

# Combine the data into a single DataFrame
data = pd.DataFrame({
    'Birmingham': B_NO2,
    'London': l_NO2,
    'Liverpool': lp_NO2
})

# Perform one-way ANOVA
f_statistic, p_value = stats.f_oneway(data['Birmingham'], data['London'], data['Liverpool'])

# Print the results
print("F-statistic:", f_statistic)
print("p-value:", p_value)

# Check for statistical significance (alpha = 0.05)
if p_value < 0.05:
    print("The means are significantly different among the cities.")
    # Perform Tukey's HSD test for post hoc analysis
    mc_result = mc.MultiComparison(data.melt(var_name='City', value_name='NO2 Concentration')['NO2 Concentration'],
                                   data.melt(var_name='City', value_name='NO2 Concentration')['City'])
    tukey_result = mc_result.tukeyhsd()
    print(tukey_result)
else:
    print("There is no significant difference among the cities.")


In [ ]:
# Combine the data frames
merged_df = df_b_NO2.merge(df_l_NO2, on='Year').merge(df_lp_NO2, on='Year')

# Create a new data frame for Seaborn (melt the data to make it long-form)
melted_df = pd.melt(merged_df, id_vars='Year', var_name='City', value_name='NO2_Concentration')

# Plot box plots using Seaborn
plt.figure(figsize=(10, 6))
sns.boxplot(x='City', y='NO2_Concentration', data=melted_df)
plt.title('Annual Mean NO2 pollutant of London, Birmingham and Liverpool')
plt.ylabel('Annual Mean of NO2')
plt.xlabel('City')
plt.grid(True)
plt.show()


In [ ]:
skewness_value = df_lp_NO2['NO2_Liverpool'].skew()

kurtosis_value = df_b_NO2['NO2_Birmingham'].kurtosis()
print("Skewness:", skewness_value)
print("Kurtosis:", kurtosis_value)

In [ ]:
plt.plot(df_lp_NO2['NO2_Liverpool'])

In [ ]:
max_lag = 10  # Example: Compute autocorrelation for lags up to 10.
autocorrelation_values = [df_b_NO2['NO2_Birmingham'].autocorr(lag=lag) for lag in range(1, max_lag+1)]

print("Autocorrelation:", autocorrelation_values)

In [ ]:
plt.figure(figsize=(10, 5))
plt.stem(range(1, max_lag+1), autocorrelation_values, use_line_collection=True)
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.title("Autocorrelation Function (ACF)")
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import f

# Assuming B_NO2, l_NO2, and lp_NO2 are pandas Series containing the data for Birmingham, London, and Liverpool, respectively.

# Combine the data into a single DataFrame
data = pd.DataFrame({
    'Birmingham': B_NO2,
    'London': l_NO2,
    'Liverpool': lp_NO2
})

# Perform one-way ANOVA
f_statistic, p_value = stats.f_oneway(data['Birmingham'], data['London'], data['Liverpool'])

# Print the results
print("F-statistic:", f_statistic)
print("p-value:", p_value)

# Check for statistical significance (alpha = 0.05)
if p_value < 0.05:
    print("The means are significantly different among the cities.")
    
    # Perform Fisher's LSD test for post hoc analysis
    k = len(data.columns)  # Number of groups (cities)
    n_total = data.shape[0]  # Total number of observations
    group_means = data.mean()  # Mean values for each city
    group_n = data.count()  # Sample sizes for each city

    # Calculate the LSD value
    lsd_value = np.sqrt(f.ppf(1 - 0.05, k - 1, n_total - k) * np.var(data.values, ddof=1) / n_total)

    # Perform all pairwise comparisons
    for i, city1 in enumerate(data.columns):
        for j, city2 in enumerate(data.columns[i+1:], i+1):
            mean_difference = group_means[city1] - group_means[city2]
            critical_value = lsd_value * np.sqrt(1/group_n[city1] + 1/group_n[city2])

            if abs(mean_difference) > critical_value:
                print(f"{city1} and {city2} have significantly different pollution levels.")
else:
    print("There is no significant difference among the cities.")


In [ ]:
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.multicomp as mc

# Assuming B_NO2, l_NO2, and lp_NO2 are pandas Series containing the data for Birmingham, London, and Liverpool respectively.

# Combine the data into a single DataFrame
data = pd.DataFrame({
    'Birmingham': B_NO2,
    'London': l_NO2,
    'Liverpool': lp_NO2
})

# Perform one-way ANOVA
f_statistic, p_value = stats.f_oneway(data['Birmingham'], data['London'], data['Liverpool'])

# Print the results
print("F-statistic:", f_statistic)
print("p-value:", p_value)

# Check for statistical significance (alpha = 0.05)
if p_value < 0.05:
    print("The means are significantly different among the cities.")
    # Perform Tukey's HSD test for post hoc analysis
    mc_result = mc.MultiComparison(data.melt(var_name='City', value_name='NO Concentration')['NO Concentration'],
                                   data.melt(var_name='City', value_name='NO Concentration')['City'])
    tukey_result = mc_result.tukeyhsd()
    print(tukey_result)

    # Get the increasing order of cities based on their NO2 concentrations
    mean_values = data.mean().sort_values(ascending=False)  # Sort in ascending order
    increasing_order = mean_values.index.tolist()
    print("Increasing order of cities based on NO2 concentrations:")
    print(increasing_order)

else:
    print("There is no significant difference among the cities.")
